In [2]:
import torch

In [32]:
from src.text_encoder import CTCTextEncoder
text_encoder = CTCTextEncoder()

In [33]:
text = 'wwwwwaaa losseer'
encoded = text_encoder.encode(text)
assert 'wa loser' == text_encoder.ctc_decode(encoded.flatten())

In [34]:
text = ''
encoded = text_encoder.encode(text)
assert '' == text_encoder.ctc_decode(encoded.flatten())

In [8]:
!pip install editdistance

In [53]:
ind2char = text_encoder.ind2char

In [59]:
def ctc_decode1(inds) -> str:
    """
    Decoding with CTC.
    """
    text = ''
    prev_symb = ''

    for index in inds:
        current = index

        if current == prev_symb:
            continue

        prev_symb = current
        text += ind2char[current]
    return text

In [60]:
def ctc_decode2(inds) -> str:
    """
    CTC decoding implementation.

    Args:
        inds (list): list of tokens.
    Returns:
        decoded_text (str): decoded text without empty tokens and
            repetitions.
    """
    decoded = []
    prev_char = None
    for ind in inds:
        cur_char = ind2char[int(ind)]
        if cur_char != '' and cur_char != prev_char:
            decoded.append(ind2char[int(ind)])
        prev_char = ind2char[int(ind)]
    return "".join(decoded).strip()

In [61]:
ctc_decode1([1,2,2,2,3,3,4,5]) == ctc_decode2([1,2,2,2,3,3,4,5])

True

In [23]:
from src.metrics.utils import calc_cer, calc_wer

In [49]:
a = torch.randn(2, 206, 28)
torch.nn.functional.log_softmax(a, dim=-1).shape

torch.Size([2, 206, 28])

In [10]:
assert calc_cer('hello', 'hallo') == 0.2
assert calc_wer('hello world', 'hallo word') == 1
assert calc_cer('', '') == 0
assert calc_wer('', '') == 0
assert calc_wer('', 'a') == 1

In [ ]:
from src.metrics.wer import BeamSearchWERMetric
beam_search_wer = BeamSearchWERMetric(text_encoder)
log_probs = torch.randn(1, 3, len(text_encoder.vocab))
log_probs_length = torch.tensor(log_probs.shape[1]).unsqueeze(0)
print(log_probs_length)

tensor([3])
aiah


1.0

In [6]:
beam_search_wer(log_probs, log_probs_length, ['aia'])

aiah


1.0

In [8]:
from src.metrics.сer import BeamSearchСERMetric
beam_search_сer = BeamSearchСERMetric(text_encoder)
log_probs = torch.randn(1, 3, len(text_encoder.vocab))
log_probs_length = torch.tensor(log_probs.shape[1]).unsqueeze(0)
print(log_probs_length)

ModuleNotFoundError: No module named 'src.metrics.сer'

In [3]:
torch.randn(1, 3, 5).squeeze(0).shape

torch.Size([3, 5])

In [4]:
a = torch.randn(1, 5)
a[0:1, :].shape

torch.Size([1, 5])

In [2]:
import torch

In [3]:
a = torch.randn(1, 5).squeeze(0)
b = torch.randn(1, 3).squeeze(0)
from torch.nn.utils.rnn import pad_sequence
pad_sequence([a, b], batch_first=True, padding_value=0).shape


torch.Size([2, 5])

In [6]:
torch.randn(2, 3, 1).transpose(1,2).shape

torch.Size([2, 1, 3])

In [19]:
t1 = torch.randn(1, 10)
t2 = torch.randn(1, 15)

In [ ]:
pad_sequence([t1.squeezeи(0), t2.squeeze(0)], batch_first=True, padding_value=0).shape

torch.Size([2, 15])

In [2]:
import torch

In [3]:
T, V = 200, 28
log_probs = torch.randn(T, V)

In [4]:
from src.metrics.utils import ctc_beam_search

In [ ]:
ctc_beam_search(log_probs, ctc_blank='')

AttributeError: 'list' object has no attribute 'shape'

In [10]:
list(tuple(tuple(tuple(tuple(1, 2), 3), 4), 5))

TypeError: tuple expected at most 1 argument, got 2

In [8]:
log_probs[:100].shape

torch.Size([100, 28])

In [15]:
list(tuple((tuple((1, 2)), 3)))

[(1, 2), 3]

In [2]:
import torch

In [89]:
dataset_items = [{'spectrogram': torch.randn(1, 3, 20), 'text_encoded': torch.randn(1, 7)}]

In [90]:
def collate_fn_levon(dataset_items: list[dict]):
    """
    Collate and pad fields in the dataset items.
    Converts individual items into a batch.

    Args:
        dataset_items (list[dict]): list of objects from
            dataset.__getitem__.
    Returns:
        result_batch (dict[Tensor]): dict, containing batch-version
            of the tensors.
    """
    if not dataset_items:
        return {}

    batch = {}

    for sample in dataset_items:
        for key, val in sample.items():
            if key not in batch:
                batch[key] = []
            batch[key].append(val)

    def pad_tensors(tensors):
        # список тензоров. Нашли макс длину
        max_len = max(t.shape[-1] for t in tensors)
        padded_tensors = []
        lengths = []
        for t in tensors:
            # текущая длина
            lengths.append(t.shape[-1])
            # сколько допаддить
            padding_size = max_len - t.shape[-1]
            padded = nn.functional.pad(t, (0, padding_size))
            padded_tensors.append(padded)
        padded_tensors = torch.concat(padded_tensors)
        lengths = torch.tensor(lengths)
        return padded_tensors, lengths

    keys = list(batch.keys())
    for key in keys:
        if key in ['audio', 'spectrogram', 'text_encoded']:
            padded, lengths = pad_tensors(batch[key])
            batch[key] = padded
            batch[key + '_length'] = lengths
        elif key in ['text', 'audio_path']:
            pass
        else:
            raise ValueError("Unexpected key `%s` encountered" % (key,))
    
    return batch

In [91]:
levon_collate = collate_fn_levon(dataset_items)

In [92]:
def collate_fn(dataset_items: list[dict]):
    """
    Collate and pad fields in the dataset items.
    Converts individual items into a batch.

    Args:
        dataset_items (list[dict]): list of objects from
            dataset.__getitem__.
    Returns:
        result_batch (dict[Tensor]): dict, containing batch-version
            of the tensors.
    """
    result_batch = {
        key: []
        for key in [
            "audio",
            "spectrogram",
            "text",
            "text_encoded",
            "audio_path",
            "spectrogram_length",
            "text_encoded_length",
        ]
    }
    if not dataset_items:
        return result_batch
    
    # print(dataset_items[0]['spectrogram'].shape, dataset_items[1]['spectrogram'].shape)
    # assert 1 == 0

    for element in dataset_items:
        # result_batch["audio"].append(element["audio"])
        # result_batch["audio_path"].append(element["audio_path"])

        result_batch["spectrogram"].append(
            element["spectrogram"].squeeze(0).transpose(0, 1) # (1, F, T) -> (T, F)
        )
        # result_batch["spectrogram_length"].append(element["spectrogram"].shape[-1])

        # result_batch["text"].append(element["text"])
        result_batch["text_encoded"].append(element["text_encoded"].transpose(-1, -2))
        # result_batch["text_encoded_length"].append(element["text_encoded"].shape[-1])

    result_batch["spectrogram"] = (
        pad_sequence(result_batch["spectrogram"], batch_first=True)
        .transpose(-1, -2)
        .contiguous()
    )
    # result_batch["spectrogram_length"] = torch.tensor(
    #     result_batch["spectrogram_length"], dtype=torch.int
    # )

    result_batch["text_encoded"] = (
        pad_sequence(result_batch["text_encoded"], batch_first=True)
        .squeeze(-1)
        .contiguous()
    )
    # result_batch["text_encoded_length"] = torch.tensor(
    #     result_batch["text_encoded_length"], dtype=torch.int
    # )

    return result_batch


In [93]:
my_collate = collate_fn(dataset_items)

In [68]:
levon_spec = levon_collate['spectrogram']

In [69]:
my_spec = my_collate['spectrogram']

In [70]:
levon_spec.shape == my_spec.shape

True

In [73]:
levon_spec

tensor([[[-1.2236, -0.7313, -0.2119, -0.6008, -1.3219],
         [-2.0201,  0.7774, -0.8372,  0.7074,  0.8578],
         [-1.2288,  2.3449, -0.0179, -2.6620,  0.5698]]])

In [74]:
my_spec

tensor([[[-1.2236, -0.7313, -0.2119, -0.6008, -1.3219],
         [-2.0201,  0.7774, -0.8372,  0.7074,  0.8578],
         [-1.2288,  2.3449, -0.0179, -2.6620,  0.5698]]])

In [ ]:
levon_enc = levon_collate['text_encoded']
levon_enc

tensor([[-0.1954, -1.7053, -1.2088,  1.7237,  0.7177, -0.2447,  1.2513]])

In [95]:
my_enc = my_collate['text_encoded']
my_enc

tensor([[-0.1954, -1.7053, -1.2088,  1.7237,  0.7177, -0.2447,  1.2513]])

In [96]:
torch.allclose(levon_enc, my_enc)

True